In [ ]:
# default_exp database

# Database Class

> API details.

In [ ]:
#hide
import logging
logging.basicConfig(level= logging.WARNING)
log = logging.getLogger("pynamodb")
log.setLevel(logging.DEBUG)
log.setLevel(logging.WARNING)
log.propagate = True


In [ ]:
#hide
import pickle, os, json

os.environ['DATABASE_TABLE_NAME'] = 'product-table-dev-manual'
os.environ['BRANCH'] = 'dev'
os.environ['REGION'] = 'ap-southeast-1'
os.environ['INVENTORY_BUCKET_NAME'] = 'product-bucket-dev-manual'
os.environ['INPUT_BUCKET_NAME'] = 'input-product-bucket-dev-manual'
os.environ['DAX_ENDPOINT'] = 'longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111'
os.environ['LINEKEY'] = '2uAfV4AoYglUGmKTAk2xNOm0aV2Ufgh1BQPvQl9vJd4'
os.environ['INTCOLS'] = json.dumps(['cprcode', 'iprcode', 'oprcode',  'sellingPrice'])
REGION = 'ap-southeast-1'

In [ ]:
#hide
print(json.dumps(os.environ['INTCOLS']))

"[\"cprcode\", \"iprcode\", \"oprcode\", \"sellingPrice\"]"


In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import pandas as pd
from datetime import datetime
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute, JSONAttribute, BooleanAttribute, BinaryAttribute
from pynamodb.indexes import GlobalSecondaryIndex, AllProjection
from botocore.config import Config
from s3bz.s3bz import S3, ExtraArgs
from pprint import pprint
from nicHelper.wrappers import add_method, add_class_method, add_static_method
from nicHelper.dictUtil import stripDict, printDict
from nicHelper.exception import errorString
from nicHelper import pdUtils
from awsSchema.apigateway import Response, Event
from dataclasses_json import dataclass_json, Undefined, CatchAll
from dataclasses import dataclass
from typing import List
from villaProductDatabase.query import Querier
from villaProductDatabase.helpers import Helpers
from villaProductDatabase.s3 import S3Cache
from villaProductDatabase.schema import KeySchema, createIndex
from villaProductDatabase.update import Updater
from requests import post

import pickle, json, boto3, bz2, requests, validators, os, logging, traceback, zlib, yaml, sys

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
#export
import os

DATABASE_TABLE_NAME = os.environ.get('DATABASE_TABLE_NAME')
INVENTORY_BUCKET_NAME = os.environ.get('INVENTORY_BUCKET_NAME')
INPUT_BUCKET_NAME = os.environ.get('INPUT_BUCKET_NAME')
REGION = os.environ.get('REGION') or 'ap-southeast-1'
ACCESS_KEY_ID = os.environ.get('USER') or None
SECRET_ACCESS_KEY = os.environ.get('PW') or None
LINEKEY= os.environ.get('LINEKEY')
BRANCH= os.environ.get('BRANCH')
DBHASHLOCATION = '/mnt/efs/database.hash'
DBCACHELOCATION = '/mnt/efs/database.cache'
INTCOLS = json.loads(os.environ.get('INTCOLS') or '{}')
ECOMMERCE_COL_LIST = f'https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/{BRANCH}/product/ecommerceColList.yaml'
  
try:
  DAX_ENDPOINT = os.environ['DAX_ENDPOINT']
except KeyError as e:
  DAX_ENDPOINT = None
  print(f'dax endpoint missing {e}')
  
print(DAX_ENDPOINT)

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


# Main Database Class

In [ ]:
#export
# dont forget to import dependent classes from the relevant notebooks
class ProductDatabase( Querier, Helpers, KeySchema, S3Cache, Updater):
  class Meta:
    aws_access_key_id = ACCESS_KEY_ID
    aws_secret_access_key = SECRET_ACCESS_KEY
    table_name = DATABASE_TABLE_NAME
    region = REGION
    billing_mode='PAY_PER_REQUEST'
    dax_read_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
    dax_write_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None


  TRUE = 'Y'
  FALSE = 'N'
  
  # indexes
  needsUpdateIndex = createIndex('needsUpdate','sellingPrice')
  cprcodeIndex = createIndex('cprcode', 'sellingPrice')
  oprcodeIndex = createIndex('oprcode', 'sellingPrice')
  pr_dpcodeIndex = createIndex('pr_dpcode', 'sellingPrice')
  pr_barcodeIndex = createIndex('pr_barcode', 'sellingPrice')
  pr_barcode2Index = createIndex('pr_barcode2', 'sellingPrice')
  pr_suref3Index = createIndex('pr_suref3', 'sellingPrice')
  pr_sa_methodIndex = createIndex('pr_sa_method', 'sellingPrice')
  
  
    
  def __repr__(self):
    return self.returnKW(self.data)
  def syncIndex(self):
    keys = {k:v for k,v in self.data if k in []}
  
    
  @staticmethod
  def returnKW(inputDict):
    outputStr = 'ProductDatabase Object\n'
    for k,v in inputDict.items():
      outputStr += f'{k} {v}\n'
    return outputStr  
  @staticmethod
  def convertIntCols(products:List[dict])->List[dict]:
    convertedProducts = [{k:int(float(v)) if k in INTCOLS else v for k,v in product.items()}for product in products]
    return convertedProducts
  
    
  
    

# set up notebook for tests

In [ ]:
%%time
#hide
## set items to udpate
with ProductDatabase.batch_write() as batch:
  for product in ProductDatabase.scan(limit=500):
    product.setUpdate(batch=batch)

CPU times: user 257 ms, sys: 15.6 ms, total: 272 ms
Wall time: 539 ms


In [ ]:
#hide
##tests
assert ProductDatabase.Meta.table_name == 'product-table-dev-manual', 'product table name not set'
assert len(list(ProductDatabase.scan(limit=1))) == 1, 'db scan not working'
assert len(list(ProductDatabase.needsUpdateIndex.query(ProductDatabase.TRUE))) > 1, 'cant find any needsUpdate item'

In [ ]:
#hide
### set test env
sampleProducts = [{'cprcode': '0171670', 'iprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAPAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845     ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0235141', 'iprcode': '0235141', 'oprcode': '0235141', 'ordertype': 'Y', 'pr_abb': 'EEBOO-PZCT3-PUZZLE', 'pr_active': 'Y', 'pr_cgcode': '08', 'pr_code': '0235141', 'pr_dpcode': '19', 'pr_engname': 'EEBOO,ANIMAL COUNTING PUZZLE_3ED,PZCT3', 'pr_ggcode': '113', 'pr_market': 'eeboo,PUZZLE-PZCT3', 'pr_name': 'EEBOO-PZCT3-ตัวต่อนับเลข ANIMAL COUNTING_3ED', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM1979    ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0217153', 'iprcode': '0217153', 'oprcode': '0217153', 'ordertype': 'Y', 'pr_abb': 'COCOA LOCO MILK CHOC', 'pr_active': 'Y', 'pr_cgcode': '98', 'pr_code': '0217153', 'pr_dpcode': '28', 'pr_engname': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.', 'pr_ggcode': '003', 'pr_market': 'COCOA LOCO MILK CHOCOLATE OWL', 'pr_name': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.', 'pr_puqty': '24', 'pr_sa_method': '1', 'pr_sucode1': 'F1222     ', 'pr_suref3': 'S', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0182223', 'iprcode': '0182223', 'oprcode': '0182223', 'ordertype': 'Y', 'pr_abb': 'CIRIO PIZZASSIMO 400', 'pr_active': 'Y', 'pr_cgcode': '06', 'pr_code': '0182223', 'pr_dpcode': '06', 'pr_engname': 'CIRIO PIZZASSIMO 400G.', 'pr_ggcode': '004', 'pr_market': 'CIRIO ซอสทำพิซซ่า 400 G.', 'pr_name': 'CIRIO ซอสทำพิซซ่า 400 G.', 'pr_puqty': '12', 'pr_sa_method': '1', 'pr_sucode1': '2589      ', 'pr_suref3': 'C', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0124461', 'iprcode': '0124461', 'oprcode': '0124461', 'ordertype': 'Y', 'pr_abb': 'NEW CHOICE LYCHEE', 'pr_active': 'Y', 'pr_cgcode': '02', 'pr_code': '0124461', 'pr_dpcode': '02', 'pr_engname': 'NEW CHOICE LYCHEE', 'pr_ggcode': '003', 'pr_market': 'NEW CHOICE กลิ่นลิ้นจี่', 'pr_name': 'NEW CHOICE กลิ่นลิ้นจี่', 'pr_puqty': '12', 'pr_sa_method': '1', 'pr_sucode1': '695       ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}]

# Update with dictionary

## test

In [ ]:
%%time
## get original item
item = next(ProductDatabase.query(9))
print(f'engname is {item.data.get("pr_engname")}')

### update item
updatedItem = ProductDatabase.updateWithDict(item,{'pr_engname':'testName'})
assert updatedItem.data.get('pr_engname') == 'testName'
print(f' name is changed to  {updatedItem.data.get("pr_engname")}')

engname is JOHNNIE WALKER RED 70 CL.
 name is changed to  testName
CPU times: user 3.43 ms, sys: 292 µs, total: 3.72 ms
Wall time: 6 ms


### Batch load and save data

In [ ]:
%%time
data = ProductDatabase.loadFromCache()
assert data.shape[0] > 10000, 'loadFrom cache not working'

loading hashkey allData-hash
loaded hash is 5ab4ab6eb4875b73fc878567db2765589e723e37
data is still in sync, using local cache
CPU times: user 308 ms, sys: 93.6 ms, total: 401 ms
Wall time: 682 ms


# Dump database to s3 cache 

In [ ]:
def resetCache():
  r = ProductDatabase.resetS3Cache(limit=50000)
  return Response.returnSuccess(r)

## check table length

In [ ]:
%%time
table = list(ProductDatabase.scan())
len(table)

CPU times: user 7.59 s, sys: 452 ms, total: 8.04 s
Wall time: 11.8 s


45075

## reset cache test

In [ ]:
%%time
resetCache()

db is scanned
(45075, 75) changes to update
hashKey is allData-hash
saving hash to s3
saved hash 9dc8d22f45ca04f053917ab08f13f4ec104dccc4
CPU times: user 14.2 s, sys: 1.01 s, total: 15.2 s
Wall time: 20.2 s


{'body': 'true',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

### CacheDb method

In [ ]:
#export
@add_class_method(ProductDatabase)
def cacheDb(cls, bucketName = INVENTORY_BUCKET_NAME, key = 'allData', limit=100, **kwargs):
  '''cache db to s3 and local efs'''
  db:pd.DataFrame = cls.loadFromCache(key=key, localCache=DBCACHELOCATION,
                                      localHash=DBHASHLOCATION, bucket=bucketName)
  logging.debug(f'origin item is shape{db.shape}')
  #### get change
  print('quering for changes')
  changes = list(cls.needsUpdateIndex.query(cls.TRUE, limit=limit))
  print('convert to df')
  changesDf = cls.toDf(changes)
  changesDf.set_index('cprcode', inplace=True)
  print(changesDf.shape)
#   db.set_index('cprcode', inplace=True)
  updatedDb = db.append(changesDf).drop_duplicates('cprcode', keep='last')
#   updatedDb.reset_index(inplace=True)
  cls.saveRemoteCache(updatedDb)
  print('saving to remote cache')
  with cls.batch_write() as batch:
    for item in changes:
      item.setNoUpdate(batch=batch)
  cls.notify(f'db shape is {db.shape}')
  return updatedDb

In [ ]:
%%time
# originalDb = ProductDatabase.loadFromCache()
# listPd = list(product.needsUpdateIndex.query('Y', limit=5))
# changesDf = ProductDatabase.toDf(listPd)
# changesDf.set_index('cprcode', inplace=True)
# originalDb.set_index('cprcode',inplace=True)
# newDf = originalDb.append(changesDf)
# newDf.reset_index(inplace=True)
# originalDb

ProductDatabase.cacheDb()

loading hashkey allData-hash
loaded hash is 9dc8d22f45ca04f053917ab08f13f4ec104dccc4
data is still in sync, using local cache
quering for changes
convert to df
(100, 58)
hashKey is allData-hash
saving hash to s3
saved hash 575c60ff28c4571b5ca0ab05ae8b64816c680f95
saving to remote cache
CPU times: user 4.02 s, sys: 470 ms, total: 4.49 s
Wall time: 6.17 s


,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_dpcode,pr_engname,pr_ggcode,...,preorder_fix_date,preorder_relative_day,meta_title,meta_keywords,meta_description,depth,priority_score,product_attribute_id,product_attribute,related_products
0,225407.0,225407,225407,Y,MOKU,True,87,21,KONJAC LINGUINI,010,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
1,241101.0,241101,241101,Y,BUMILGOCHUJANG,True,06,08,BUMILGOCHUJANG,006,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
2,190100.0,190100,190100,Y,WHITE SHIMEJI,True,05,19,WHITE SHIMEJI,143,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
3,62644.0,62644,62644,Y,STUTE APRICOT JAM430,True,07,08,STUTE APRICOT JAM 430 G.,002,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
4,192167.0,192167,192167,Y,**COLGATE TOTAL CHAR,True,10,09,**COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,005,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45071,76931.0,76931,76931,Y,ST. REMY BRANDY 70CL,True,16,01,ST. REMY BRANDY 70CL.,002,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
45072,203391.0,203391,203391,Y,CH TERREFORT-QUANCA,True,10,01,CH TERREFORT-QUANCARD,001,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
45073,205603.0,205603,205603,Y,19 GRAIN WHEAT LOAF,True,15,19,19 GRAIN WHEAT LOAF,132,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
45074,211585.0,211585,211585,Y,EGG,True,49,19,EGG,001,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None


### dump to s3 method

In [ ]:
@add_class_method(ProductDatabase)
def dumpToS3(cls, bucketName= INVENTORY_BUCKET_NAME, key = 'allData', limit=500, **kwargs):
  return cls.cacheDb(limit=1000)


In [ ]:
%%time
ProductDatabase.dumpToS3()

loading hashkey allData-hash
loaded hash is 575c60ff28c4571b5ca0ab05ae8b64816c680f95
data is still in sync, using local cache
quering for changes
convert to df
(1000, 58)
hashKey is allData-hash
saving hash to s3
saved hash 2c173546efe68a494d4484423b33e1b2e8de6d03
saving to remote cache
CPU times: user 4.38 s, sys: 496 ms, total: 4.88 s
Wall time: 7.21 s


,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_dpcode,pr_engname,pr_ggcode,...,preorder_fix_date,preorder_relative_day,meta_title,meta_keywords,meta_description,depth,priority_score,product_attribute_id,product_attribute,related_products
0,225407.0,225407,225407,Y,MOKU,True,87,21,KONJAC LINGUINI,010,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
1,241101.0,241101,241101,Y,BUMILGOCHUJANG,True,06,08,BUMILGOCHUJANG,006,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
2,190100.0,190100,190100,Y,WHITE SHIMEJI,True,05,19,WHITE SHIMEJI,143,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
3,62644.0,62644,62644,Y,STUTE APRICOT JAM430,True,07,08,STUTE APRICOT JAM 430 G.,002,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
4,192167.0,192167,192167,Y,**COLGATE TOTAL CHAR,True,10,09,**COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,005,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45071,76931.0,76931,76931,Y,ST. REMY BRANDY 70CL,True,16,01,ST. REMY BRANDY 70CL.,002,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
45072,203391.0,203391,203391,Y,CH TERREFORT-QUANCA,True,10,01,CH TERREFORT-QUANCARD,001,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
45073,205603.0,205603,205603,Y,19 GRAIN WHEAT LOAF,True,15,19,19 GRAIN WHEAT LOAF,132,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
45074,211585.0,211585,211585,Y,EGG,True,49,19,EGG,001,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None


### lambda function dump to s3

In [ ]:
#export
def lambdaDumpToS3(event, _):
  result = ProductDatabase.cacheDb(limit = 500)
  lastItem = result.iloc[0].to_json()
  return Response.returnSuccess({'result': json.loads(lastItem)})
#   print(result.fillna(''))
#   return Response.getReturn(body = {'result': result.fillna('').iloc[0].to_dict()})

In [ ]:
%%time
result = Response.fromDict(lambdaDumpToS3('','')).body['result']
print(result)

loading hashkey allData-hash
loaded hash is 2c173546efe68a494d4484423b33e1b2e8de6d03
data is still in sync, using local cache
quering for changes
convert to df
(500, 58)
hashKey is allData-hash
saving hash to s3
saved hash 6c2f587eca3f1e4b484acd6f07dfba9a1152c155
saving to remote cache
{'cprcode': 225407.0, 'iprcode': 225407, 'oprcode': 225407, 'ordertype': 'Y', 'pr_abb': 'MOKU', 'pr_active': True, 'pr_cgcode': '87', 'pr_dpcode': '21', 'pr_engname': 'KONJAC LINGUINI', 'pr_ggcode': '010', 'pr_market': 'MOKU', 'pr_name': 'บุกเส้นแบน ตราโมกุ', 'pr_sa_method': '1', 'pr_sucode1': 'F1148', 'pr_suref3': 'A', 'prtype': 'I', 'pstype': '1', 'pr_country_th': '', 'pr_country_en': 'Thailand', 'pr_keyword_th': '', 'pr_keyword_en': '', 'pr_filter_th': '', 'pr_filter_en': '', 'online_category_l1_th': 'อาหารสำเร็จรูปและไข่', 'online_category_l1_en': 'Delicatessen & Eggs', 'online_category_l2_th': 'อาหารพร้อมทาน', 'online_category_l2_en': 'Ready Meals', 'online_category_l3_th': '', 'online_category_l3_e

In [ ]:
product.loadFromCache()

loading hashkey allData-hash
loaded hash is 6c2f587eca3f1e4b484acd6f07dfba9a1152c155
data is still in sync, using local cache


,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_dpcode,pr_engname,pr_ggcode,...,preorder_fix_date,preorder_relative_day,meta_title,meta_keywords,meta_description,depth,priority_score,product_attribute_id,product_attribute,related_products
0,225407.0,225407,225407,Y,MOKU,True,87,21,KONJAC LINGUINI,010,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
1,241101.0,241101,241101,Y,BUMILGOCHUJANG,True,06,08,BUMILGOCHUJANG,006,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
2,190100.0,190100,190100,Y,WHITE SHIMEJI,True,05,19,WHITE SHIMEJI,143,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
3,62644.0,62644,62644,Y,STUTE APRICOT JAM430,True,07,08,STUTE APRICOT JAM 430 G.,002,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
4,192167.0,192167,192167,Y,**COLGATE TOTAL CHAR,True,10,09,**COLGATE TOTAL CHARCOAL DEEP CLEAN 150G,005,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45071,76931.0,76931,76931,Y,ST. REMY BRANDY 70CL,True,16,01,ST. REMY BRANDY 70CL.,002,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
45072,203391.0,203391,203391,Y,CH TERREFORT-QUANCA,True,10,01,CH TERREFORT-QUANCARD,001,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
45073,205603.0,205603,205603,Y,19 GRAIN WHEAT LOAF,True,15,19,19 GRAIN WHEAT LOAF,132,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None
45074,211585.0,211585,211585,Y,EGG,True,49,19,EGG,001,...,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None


# filter and cache for ecommerce

In [ ]:
#export
def lambdaDumpOnlineS3(event, *args):
  print(f'ecommece col list is {ECOMMERCE_COL_LIST}')
  # get all products from db
  df:pd.DataFrame = pd.DataFrame([i.data for i in ProductDatabase.scan()])
  # get online list from ECOMMERCE_COL_LIST
  onlineList:List[str] = yaml.load(requests.get(ECOMMERCE_COL_LIST).content)
  # filter df for item
  ## condition 1 master online is true
  condition1 = df['master_online'] == True
  ## condition 2 hema_name_en is not blank
  condition2 = df['hema_name_en'] != ''
  ## filtered df
  onlineDf:pd.DataFrame = df[condition1 & condition2].loc[:,onlineList]
  ### log shape and size
  print('shape is:',onlineDf.shape)
  print('size is:',sys.getsizeof(onlineDf.to_json(orient='split'))/1e6, 'Mb')
  
  # save file as gzip
  key = 'onlineData'
  bucket = INVENTORY_BUCKET_NAME
  path = '/tmp/inventory.json'
  ## export to gzip
  onlineDf.to_json(path ,orient='split',compression='gzip')
  ## upload file to s3
  S3.saveFile(key=key,path=path,bucket=bucket,
            ExtraArgs = {**ExtraArgs.gzip, **ExtraArgs.publicRead })
  return Response.returnSuccess()



## test

In [ ]:
lambdaDumpOnlineS3({})

<ipython-input-26-6f20c99e62c6>:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  onlineList:List[str] = yaml.load(requests.get(ECOMMERCE_COL_LIST).content)


shape is: (19108, 11)
size is: 3.272721 Mb


# Save using Standard (instant save)

In [ ]:
#export
@dataclass_json(undefined=Undefined.INCLUDE)
@dataclass
class Product:
  iprcode: int
  cprcode: int
  data: CatchAll
@dataclass_json
@dataclass
class ValueUpdate:
  items: List[Product]

In [ ]:
#export
def chunks(l, n): return [l[x: x+n] for x in range(0, len(l), n)]

In [ ]:
chunks(list(range(100)),10)

In [ ]:
sampleProducts = [{ 'cprcode': '0171670', 'iprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}] #ProductDatabase.valueUpdate({'items':sampleProducts})
ProductDatabase.valueUpdate2({'items':ProductDatabase.convertIntCols(sampleProducts)})

In [ ]:
#export
def lambdaUpdateProduct (event, _):
  products = Event.parseBody(event)['products']
  convertedProducts = ProductDatabase.convertIntCols(products)
  result = ProductDatabase.valueUpdate2({'items':convertedProducts})
  return Response.getReturn(body = result)

In [ ]:
event = Event.getInput(body = {'products':sampleProducts})
lambdaUpdateProduct(event, '')

## Save using s3

### update using s3 link

In [ ]:
#export
@add_class_method(ProductDatabase)
def updateS3Input(cls, inputBucketName:str = INPUT_BUCKET_NAME, key:str = '', **kwargs):
  products = S3.load(key=key, bucket = inputBucketName,  **kwargs)
  updateResult = cls.valueUpdate2({'items':cls.convertIntCols(products)})
  return updateResult


In [ ]:
inputKeyName = 'input-data-name'
saveResult = S3.save(key=inputKeyName, 
                     objectToSave = sampleProducts , 
                     bucket = INPUT_BUCKET_NAME,
                     accelerate = True)
logging.info('test input data saved to s3')
updateResult = ProductDatabase.updateS3Input( inputBucketName=INPUT_BUCKET_NAME, key= inputKeyName)

logging.info(f's3 save result is {saveResult} update result is {updateResult}')

In [ ]:
#export
def lambdaUpdateS3(event, _):
  inputKeyName = Event.from_dict(event).key()
  try:
    updateResult = ProductDatabase.updateS3Input(
      inputBucketName=INPUT_BUCKET_NAME, key= inputKeyName)
  except:
    ProductDatabase.notify(f'error updating with s3 {errorString()}')
    return Response.returnError(errorString())
  
  
  
  ProductDatabase.notify(f'success update {updateResult}')
  return Response.getReturn(body = updateResult)

In [ ]:
inputKeyName = 'input-data-name'
saveResult = S3.save(key=inputKeyName, 
                     objectToSave = sampleProducts , 
                     bucket = INPUT_BUCKET_NAME)
event = Event.getInput({'key': inputKeyName})
lambdaUpdateS3(event, '')

### error data test

In [ ]:
import yaml
testData = {'cprcode': 36679, 'iprcode': 36679, 'oprcode': 36679, 'ordertype': 'Y', 'pr_abb': 'LA COOKIES SMALL100G', 'pr_active': 'Y', 'pr_cgcode': '01', 'pr_code': '0036679', 'pr_dpcode': '18', 'pr_engname': 'LA COOKIES BAG SMALL 100 G.', 'pr_ggcode': '006', 'pr_market': 'แอลเอคุกกี้ถุงเล็ก100ก.', 'pr_name': 'แอลเอ คุกกี้ถุงเล็ก 100 ก.', 'pr_puqty': '1.00', 'pr_sa_method': '1', 'pr_sucode1': 'F106', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1', 'pr_country_th': 'nan', 'pr_country_en': 'nan', 'pr_keyword_th': 'เบเกอรี่', 'pr_keyword_en': 'Bakery', 'pr_filter_th': 'Bakery  Snacks,Fresh', 'pr_filter_en': 'Bakery  Snacks,Fresh', 'online_category_l1_th': 'อาหารสด', 'online_category_l1_en': 'Fresh', 'online_category_l2_th': 'เบเกอรี่ และ แสน็ค', 'online_category_l2_en': 'Bakery & Snacks', 'online_category_l3_th': 'nan', 'online_category_l3_en': 'nan', 'villa_category_l1_en': 'Fresh', 'villa_category_l2_en': 'Fresh Produce', 'villa_category_l3_en': 'Bakery', 'villa_category_l4_en': 'Cookie and confectionery', 'content_en': '0036679 La Bakery Cookies (s) 100gm Bakery', 'content_th': 'แอลเอ คุกกี้ถุงเล็ก 100 ก. เบเกอรี่', 'hema_brand_th': 'nan', 'hema_brand_en': 'nan', 'hema_sizedesc': 'nan', 'pr_brand_en': 'L.A. Bakery', 'pr_brand_th': 'L.A. Bakery', 'pr_online_name_en': 'La Bakery Cookies (s) 100gm', 'pr_online_name_th': 'แอลเอ คุกกี้ถุงเล็ก 100 ก.', 'hema_name_en': 'La Bakery Cookies (s) 100gm', 'hema_name_th': 'แอลเอ คุกกี้ถุงเล็ก 100 ก.', 'pr_name_en': 'LA COOKIES BAG SMALL 100 G.', 'pr_name_th': 'แอลเอ คุกกี้ถุงเล็ก 100 ก.', 'sort_weight': '0'}
print(yaml.dump([testData],allow_unicode=True))
print(yaml.dump(sampleProducts,allow_unicode=True))

In [ ]:
inputKeyName = 'input-data-name'
saveResult = S3.save(key=inputKeyName, 
                     objectToSave = [testData] , 
                     bucket = INPUT_BUCKET_NAME)
event = Event.getInput({'key': inputKeyName})
lambdaUpdateS3(event, '')

In [ ]:
ProductDatabase.valueUpdate2({'items':ProductDatabase.convertIntCols([testData])})

In [ ]:
next(ProductDatabase.query(36679))

## Query test

### Product Query

In [ ]:
#export
@dataclass_json
@dataclass
class ProductsFromList:
  iprcodes: List[str]

In [ ]:
#export
def lambdaProductsFromList(event, *args):
  productsFromList = Event.parseDataClass(ProductsFromList,event)
  result:pd.DataFrame = ProductDatabase.productsFromList(productsFromList.iprcodes)
  results:List[ProductDatabase] = ProductDatabase.fromDf(result)
  resultDicts:List[dict] = ProductDatabase.toListDict(results)
  return Response.returnSuccess(resultDicts)

In [ ]:
%%time
event = Event.getInput({'iprcodes': ['0217153','203915','0000009']})
response = lambdaProductsFromList(event)
result = Response.parseBody(response)
assert len(result) == 3, f'result length is not 3 {len(result)}'
assert type(result[0]) == dict, 'result type is not dict'

In [ ]:
event = Event.getInput({'iprcodes':['0000009','203915','0217153']})
lambdaProductsFromList(event)

In [ ]:
sampleQueryInput = {
    'iprcode': '0171670'
}  

In [ ]:
try:
  print(ProductDatabase.singleProductQuery({'iprcode':'12345'}))
except Exception as e:
  print(e)

In [ ]:
ProductDatabase.singleProductQuery(sampleQueryInput)

In [ ]:
#export
def lambdaSingleQuery(event, _):
  key, value = Event.from_dict(event).firstKey()
  try:
    result = ProductDatabase.singleProductQuery({key:value}).to_dict()
  except Exception as e:
    return Response.returnError(f'{e}')
  return Response.returnSuccess(body = result)

In [ ]:
event = Event(body = json.dumps(sampleQueryInput)).to_dict()
lambdaSingleQuery(event,'')

In [ ]:
result = lambdaSingleQuery(Event.getInput({'iprcode':'11234'}),'')
Response.parseBody(result)

### AllQuery

In [ ]:
from s3bz.s3bz import Requests
import zlib
%time url = ProductDatabase.allQuery(bucket = INVENTORY_BUCKET_NAME, key='allData-json.zl')
print(url)
# response = requests.get(url)
def retrieveJsonZl(url):
  response:requests.Response = requests.get(url)
  if response.status_code != 200:
    print(response.json())
  else:
    content = response.content
    jsonDb = zlib.decompress(content).decode()
    newDb = pd.read_json(jsonDb,orient='split')
    return newDb
%time newDb = retrieveJsonZl(url)
newDb.head()

In [ ]:
items = filter(lambda x: x != '', newDb['online_category_l1_en'])
len(list(newDb['online_category_l1_en'])) - len(list(items))

In [ ]:
items = filter(lambda x: x != '', newDb['online_category_l1_en'])
len(list(items))

In [ ]:
newDb.columns

In [ ]:
newDb['pr_suref3'].value_counts()

In [ ]:
#export
def lambdaAllQuery(event, *args):
  url = ProductDatabase.allQuery(bucket = INVENTORY_BUCKET_NAME, key='allData-json.zl')
  return Response.getReturn(body = {'url': url})

In [ ]:
from s3bz.s3bz import Requests
%time url = Response.fromDict(lambdaAllQuery('', '')).body['url']
%time newDb = retrieveJsonZl(url)
newDb.shape

In [ ]:
# Filter and upload for Ecommerce

#### AllQueryFeather

In [ ]:
#export
def lambdaAllQueryFeather(event, *args):
  key = 'allData'
  bucket = INVENTORY_BUCKET_NAME
  url = ProductDatabase.allQuery(bucket = INVENTORY_BUCKET_NAME, key=key)
  hashCode = pdUtils.loadRemoteHash(key=key, bucket=bucket, useUrl = True)
  return Response.getReturn(body = {'url': url, 'hash': hashCode})

In [ ]:
%time r = lambdaAllQueryFeather('')
%time result = Response.fromDict(r).body
url = result['url']
%time pd.read_feather(url)

In [ ]:
database = pd.read_feather(result['url'])
assert pdUtils.getDfHash(database) == result['hash'] , 'database and hash are not the same'

## Update s3, checking scan time

In [ ]:
%%time
print('checking scan time')
len(list(ProductDatabase.scan()))

In [ ]:
from nicHelper.schema import validateUrl

In [ ]:
input_ = {'cprcode': 123, 'iprcode': 123}
result = validateUrl('https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/master/Product.json', input_)

In [ ]:
result.cprcode
result.iprcode

In [ ]:
from nicHelper.dictUtil import genSchema
import yaml

In [ ]:
schema = genSchema(input_,format_='yaml')
print(schema)

In [ ]:
schemaDict = yaml.load(schema)

In [ ]:
import json
json.dumps(schemaDict)

In [ ]:
import os
os.listdir('/tmp/')

In [ ]:
import pandas as pd
pd.read_json('/tmp/product.json',orient = 'records')

In [ ]:
with open ('/tmp/product.json', 'r') as f:
  productJson = json.load(f)